In [1]:
# import packages
import pandas as pd
import numpy as np
import plotly.express as px
import os

In [2]:
# define desired directory path
directory_path = "/Users/treypallace/Desktop/Fourth Year Fall 2023/ds 4002/Project_Data/linkedin"

# use os.chdir() to change the working directory
os.chdir(directory_path)

# read in the data
job_postings_df = pd.read_csv("job_postings.csv")
job_skills_df = pd.read_csv("job_skills.csv")


### Join the job_postings_df with the job_skills_df

In [3]:
# merge data
df = job_postings_df.merge(job_skills_df, on='job_id',  how='left')

In [4]:
# select columns
df = df[['job_id', 'description', 'max_salary', 'med_salary', 'min_salary', 'pay_period', 'currency', 'work_type','compensation_type','skill_abr', 'location', 'title']]

In [5]:
# convert every salary to hourly pay period 
def convert_to_hourly(row):
    if row['pay_period'] == 'YEARLY':
        if row['work_type'] in ['FULL_TIME', 'CONTRACT']:
            return row['min_salary'] / (40 * 52), row['max_salary'] / (40 * 52)
        elif row['work_type'] == 'PART_TIME':
            return row['min_salary'] / (20 * 52), row['max_salary'] / (20 * 52)
    elif row['pay_period'] == 'MONTHLY':
        if row['work_type'] in ['FULL_TIME', 'CONTRACT']:
            return row['min_salary'] * 12 / (40 * 52), row['max_salary'] * 12 / (40 * 52)
        elif row['work_type'] == 'PART_TIME':
            return row['min_salary'] * 12 / (20 * 52), row['max_salary'] * 12 / (20 * 52)
    else:
        return row['min_salary'], row['max_salary']

# Apply the function to the DataFrame using a lambda function
df[['min_salary', 'max_salary']] = df.apply(lambda row: convert_to_hourly(row), axis=1, result_type='expand')


In [6]:
# calculate average salary
df['avg_salary'] = (df['max_salary'] + df['min_salary']) / 2

In [7]:
nan_counts = df.isna().sum()
nan_counts

job_id                   0
description              2
max_salary           17903
med_salary           25826
min_salary           17903
pay_period           16069
currency             16069
work_type                0
compensation_type    16069
skill_abr              637
location                 0
title                    0
avg_salary           17903
dtype: int64

In [8]:
# filter to remove rows without descriptions or avg_salary
df = df.dropna(subset=['description', 'avg_salary'])

## Model Building: Frequency Count 

In [9]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import re
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Download NLTK resources (if not already downloaded)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize the WordNet lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(nltk.corpus.stopwords.words('english'))

# Function to tokenize a sentence into words
def sentence_to_wordlist(raw_sentence):
    clean_sentence = re.sub("[^a-zA-Z0-9]", " ", raw_sentence)
    tokens = nltk.word_tokenize(clean_sentence)
    return tokens

# Tokenize a description into a list of word lists and remove stopwords
def tokenize(description):
    stripped_description = description.strip()
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(stripped_description)
    
    tokenized_sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            tokens = sentence_to_wordlist(raw_sentence)
            tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
            tokenized_sentences.append(tokens)
    
    return tokenized_sentences

# Get Bag of Words (BOW) counts
def get_count_vectors(descriptions):
    vectorizer = CountVectorizer(max_features=10000, ngram_range=(1, 3), stop_words='english')
    count_vectors = vectorizer.fit_transform(descriptions)
    feature_names = vectorizer.get_feature_names_out()
    return feature_names, count_vectors

# Assuming you have a DataFrame 'df' with a 'description' column
feature_names_cv, count_vectors = get_count_vectors(df['description'])


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/treypallace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/treypallace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/treypallace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Most Frequent Words before Extracting for Skills 

In [10]:
# Sum the counts of each word across all documents
word_counts = count_vectors.sum(axis=0)

# Convert the word counts to a list of (word, count) tuples
word_counts_tuples = [(word, count) for word, count in zip(feature_names_cv, word_counts.tolist()[0])]

# Sort the tuples by count in descending order
sorted_word_counts = sorted(word_counts_tuples, key=lambda x: x[1], reverse=True)

# Get the top N most frequent words (adjust N as needed)
top_N_words = 30  # Change this value to get a different number of top words
top_words = sorted_word_counts[:top_N_words]

# Print the top words with their counts
for word, count in top_words:
    print(f'{word}: {count}')


experience: 35388
work: 29388
team: 22440
skills: 17717
business: 17483
including: 16687
management: 14778
ability: 14377
job: 13689
position: 13343
benefits: 12549
time: 12439
company: 12306
years: 11944
support: 11742
required: 11501
data: 11372
role: 11211
new: 10669
development: 10483
customer: 10374
sales: 10043
knowledge: 9746
opportunity: 9675
working: 9610
project: 9339
information: 9179
range: 9096
related: 9009
status: 8561


## Extract for Skills - Named Entity Recognition 


In [12]:
 #!pip install skillNer
 #!python -m spacy download en_core_web_lg

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 3.7 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 339.7/339.7 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 784.9/784.9 kB 17.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.1/491.1 kB 11.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.5/374.5 kB 5.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.8/127.8 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [11]:
# imports
import spacy
import en_core_web_lg
from spacy.matcher import PhraseMatcher

# import skill extractor
from skillNer.skill_extractor_class import SkillExtractor
from skillNer.general_params import SKILL_DB

# init params of skill extractor
nlp = en_core_web_lg.load()
# init skill extractor
skill_extractor = SkillExtractor(nlp, SKILL_DB, PhraseMatcher)

loading full_matcher ...
loading abv_matcher ...
loading full_uni_matcher ...
loading low_form_matcher ...
loading token_matcher ...


In [12]:
#tests a random job description
import random
job_description = df['description'].sample().values[0]

# Extract skills 
annotations = skill_extractor.annotate(job_description)

skill_extractor.describe(annotations)

/Users/treypallace/miniconda3/lib/python3.10/site-packages/skillNer/utils.py:99: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  vec_similarity = token1.similarity(token2)


In [14]:
# Extract values from 'doc_node_value' and save them into a list
values_list = []
for item in annotations['results']['full_matches']:
    values_list.append(item['doc_node_value'])
    
# remove repeats
skills_list = list(set(values_list))

print(skills_list)

['operating system',
 'device management',
 'mobile device',
 'mobile device management',
 'device management',
 'application packaging',
 'production system',
 'directory service',
 'technical documentation',
 'functional specification',
 'technical support',
 'office 365 exchange online',
 'user account',
 'network management',
 'active directory',
 'user directory',
 'directory structure',
 'group policy',
 'functional specification']

In [24]:

# Calculate the number of rows in each sub-DataFrame
num_rows = len(df)
split_size = num_rows // 4

# Split the DataFrame into four equal parts
df1 = df.iloc[:split_size].copy()
df2 = df.iloc[split_size:2*split_size].copy()
df3 = df.iloc[2*split_size:3*split_size].copy()
df4 = df.iloc[3*split_size:].copy()

# Reset the index for each sub-DataFrame
df1.reset_index(drop=True, inplace=True)
df2.reset_index(drop=True, inplace=True)
df3.reset_index(drop=True, inplace=True)
df4.reset_index(drop=True, inplace=True)

# Now, df1, df2, df3, and df4 are four equal DataFrames with original indices preserved


In [20]:

# Calculate the number of rows in each sub-DataFrame
num_rows = len(df)
split_size = num_rows // 4

# Split the DataFrame into four equal parts
df_parts = np.array_split(df, 4)

# Create four DataFrames with original indices preserved
df1, df2, df3, df4 = [part.transpose().reset_index(drop=True).transpose() for part in df_parts]


In [25]:
df1

,job_id,description,max_salary,med_salary,min_salary,pay_period,currency,work_type,compensation_type,skill_abr,location,title,avg_salary
0,85008768,While many industries were hurt by the last fe...,25.000000,NaN,22.000000,YEARLY,USD,FULL_TIME,BASE_SALARY,SALE,"Chico, CA",Licensed Insurance Agent,23.500000
1,85008768,While many industries were hurt by the last fe...,25.000000,NaN,22.000000,YEARLY,USD,FULL_TIME,BASE_SALARY,BD,"Chico, CA",Licensed Insurance Agent,23.500000
2,903408693,Provide clerical and administrative support to...,20.192308,NaN,17.788462,YEARLY,USD,FULL_TIME,BASE_SALARY,ADM,"Albany, GA",Office Associate,18.990385
3,1029078768,United Staffing Solutions is partnering with o...,50.000000,NaN,50.000000,HOURLY,USD,PART_TIME,BASE_SALARY,HCPR,"Muskegon, MI",Registered Nurse (RN) Vaccinator,50.000000
4,1535492735,We are looking for a responsible Administrativ...,20.000000,NaN,18.000000,HOURLY,USD,PART_TIME,BASE_SALARY,ADM,"Ocoee, FL",Administrative Assistant,19.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2420,3693069050,Responsibilities And Duties\nProject Lead – Th...,28.846154,NaN,26.442308,YEARLY,USD,FULL_TIME,BASE_SALARY,MRKT,"Albany, NY",Scoring Content Specialist,27.644231
2421,3693069050,Responsibilities And Duties\nProject Lead – Th...,28.846154,NaN,26.442308,YEARLY,USD,FULL_TIME,BASE_SALARY,PR,"Albany, NY",Scoring Content Specialist,27.644231
2422,3693069050,Responsibilities And Duties\nProject Lead – Th...,28.846154,NaN,26.442308,YEARLY,USD,FULL_TIME,BASE_SALARY,WRT,"Albany, NY",Scoring Content Specialist,27.644231
2423,3693069051,Responsibilities And Duties\nProject Lead – Th...,28.846154,NaN,26.442308,YEARLY,USD,FULL_TIME,BASE_SALARY,MRKT,"Jackson, MS",Scoring Content Specialist,27.644231


In [26]:
## ****takes a VERY long time to run, just import the extracted_skills.csv dataframe**** 

def extract_skills(description, row_index):
    try:
        # Extract skills
        annotations = skill_extractor.annotate(description)

        # Extract values from 'doc_node_value' and save them into a list
        values_list = []
        for item in annotations['results']['full_matches']:
            values_list.append(item['doc_node_value'])

        # Remove duplicates
        skills_list = list(set(values_list))

        return skills_list
    except Exception as e:
        # Handle any exceptions (e.g., if the extraction process fails)
        print(f"Error extracting skills from description at row {row_index}: {e}")
        return []





In [27]:
# Apply the function to every description in the DataFrame and create a new column
df1['skills_extracted'] = df1.apply(lambda row: extract_skills(row['description'], row.name), axis=1)

Error extracting skills from description at row 0: list index out of range
Error extracting skills from description at row 1: list index out of range


/Users/treypallace/miniconda3/lib/python3.10/site-packages/skillNer/utils.py:99: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  vec_similarity = token1.similarity(token2)


Error extracting skills from description at row 15: list index out of range
Error extracting skills from description at row 16: list index out of range
Error extracting skills from description at row 83: 'result' is not in list
Error extracting skills from description at row 84: 'result' is not in list
Error extracting skills from description at row 142: 'stand' is not in list
Error extracting skills from description at row 183: list index out of range
Error extracting skills from description at row 247: list index out of range
Error extracting skills from description at row 248: list index out of range
Error extracting skills from description at row 249: list index out of range
Error extracting skills from description at row 308: list index out of range
Error extracting skills from description at row 309: list index out of range
Error extracting skills from description at row 310: list index out of range
Error extracting skills from description at row 550: list index out of range
Erro

In [28]:
df1.to_csv('extracted_skills1.csv', index=False)

In [ ]:
df2['skills_extracted'] = df2.apply(lambda row: extract_skills(row['description'], row.name), axis=1)

/Users/treypallace/miniconda3/lib/python3.10/site-packages/skillNer/utils.py:99: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  vec_similarity = token1.similarity(token2)


In [ ]:
df2.to_csv('extracted_skills2.csv', index=False)
# need to inspect why the descriptions at these indexes are not working

In [ ]:

# remove rows with empty skills lists 

# export the DataFrame to a CSV file
